<a href="https://colab.research.google.com/github/JuanSpecht/PDI2021/blob/main/TP4/PDI_TP4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Trabajo Práctico 4: Histograma de luminancia

### Importo bibliotecas

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from google.colab import files
from PIL import Image
from io import BytesIO
import ipywidgets as widgets
import IPython.display as ipd

### Defino las funciones

In [2]:
# Para transformar de RGB a YIQ
def rgb_to_yiq (img_rgb):

  # Defino la matriz de transformación RGB -> YIQ
  rgb2yiq = np.array([[0.299, 0.587, 0.114],
                      [0.595716, -0.274453, -0.321263],
                      [0.211456, -0.522591, 0.311135]]
                     )
  
  # Normalizo al intervalo [0,1] y transformo
  img_rgb_norm = img_rgb/255
  img_yiq = np.einsum('kl,ijl->ijk', rgb2yiq, img_rgb_norm)

  return img_yiq

# Para transformar de YIQ a RGB
def yiq_to_rgb (img_yiq):

  # Defino la matriz de transformación YIQ -> RGB
  yiq2rgb = np.array([[1, 0.9663, 0.6210],
                      [1, -0.2721, -0.6474],
                      [1, -1.1070, 1.7046]]
                     )

  # Transformo de YIQ a RGB normalizado
  img_rgb_norm = np.einsum('kl,ijl->ijk', yiq2rgb, img_yiq)

  # Acoto los valores de los canales R, G y B al intervalo [0,1]
  img_rgb_norm = np.clip(img_rgb_norm, 0, 1)

  # Convierto a bytes y redondeo a enteros
  img_rgb = img_rgb_norm * 255 
  img_rgb = np.rint(img_rgb).astype(np.int32)

  return img_rgb

# Para generar el histograma de luminancia
def histogram (img_bytes, n_bins):

    # Cargo la imagen y la paso a YIQ
    img = np.array(Image.open(BytesIO(img_bytes)))
    img_yiq = rgb_to_yiq(img)

    # Defino parámetros para generar histograma
    bins_vec = np.zeros(n_bins) # Vector con los valores de cada bin
    step = 1/len(bins_vec) # Ancho de los bines
    bins_pos = np.arange(0, 1, step) # Posición de los centros de las barras
    width = 0.8*step # Ancho de cada barra
    y_values = np.ravel(img_yiq[:,:,0]) # Canal de luminancia

    # cuento la cantidad de pixeles que corresponden a cada bin
    pixel_counter = 0
    for pixel in y_values:
        bin = np.int(np.floor(pixel*n_bins))
        if bin == n_bins:
            bin -= 1
        bins_vec[bin] += 1

    # Calculo las frecuencias relativas porcentuales
    bins_vec = 100 * bins_vec / len(y_values)
    fig, ax = plt.subplots()
    fig.tight_layout()

    # Genero al figura con el histograma y la configuro 
    ax.bar(bins_pos + step/2, bins_vec, width, align='center', color='black')

    ax.set_axisbelow(True)
    figFormat (fig, ax, 400, 160, 2, 'black')
    plt.grid()
    ax.set_xlabel('Luminancia', fontsize=12, fontweight='bold')
    ax.set_ylabel('Frec. relativa (%)', fontsize=12, fontweight='bold')
    plt.xlim([0, 1])
    plt.ylim([0, 100])
    plt.xticks(np.arange(0, 1.1, step=0.1), weight='bold')
    plt.yticks(weight='bold')
    plt.close()

    return fig, ax

# Defino la función lineal a trozos
def piecewise_linear(x, y_min, y_max):

    return np.piecewise(x, [x < y_min, (x>=y_min) & (x<=y_max) ,x> y_max], 
                        [lambda x:0, 
                        lambda x: x / (y_max - y_min) + y_min / (y_min - y_max),
                        lambda x:1])

# Función para configurar el tamaño y el borde de los gráficos
def figFormat (fig, ax, pix_width, pix_height, linewidth, color):

    for axis in ['top','bottom','left','right']:
        ax.spines[axis].set_linewidth(2)
        ax.spines[axis].set_color("black")
        ax.spines[axis].set_zorder(0)

    fig.set_size_inches(pix_width / fig.dpi, pix_height / fig.dpi)

### Código relativo a la interfaz gráfica

In [ ]:
class ModelUI:

    def __init__(self):

        self.par = dict()
        self.initUI()

    def initUI(self):

        #Creo la caja donde van los sliders
        self.slider = widgets.Box(layout=widgets.Layout(
            height='50px',
            align_items='center',))

        # Inicializo las funciones que manejan los widgets
        self.initLinear()
        self.initPower()
        self.initScalarMult()
        self.initHistDropdown()
        self.initFuncDropdown()
        self.initLoadButton()
        self.initSaveButton() 
        self.initRunButton()
        self.initFuncOutput()
        self.initInputImage()   
        self.initInputHist()
        self.initOutputImage()
        self.initOutputHist()

        # Armo el layout de la interfaz
        self.controls = widgets.VBox([
            widgets.HBox([self.load_button, self.save_button],
                         layout=widgets.Layout(justify_content='center')),
            widgets.Label(value="Número de barras del histograma:"),
            widgets.HBox([self.hist_dropdown],
                         layout=widgets.Layout(justify_content='center')),
            widgets.Label(value="Seleccione la función:"),
            widgets.HBox([self.func_dropdown],
                         layout=widgets.Layout(justify_content='center')
                         ),
            self.slider,
            widgets.VBox([self.func_output, self.run_button],
                         layout=widgets.Layout(align_items='center')
                         )
            ],
            layout=widgets.Layout(width='320px',align_items='stretch')
            )

        self.input_panel = widgets.VBox([
            self.input_image,
            widgets.Box([self.input_hist],
                        layout=widgets.Layout(overflow_x='hidden',
                                              overflow_y='hidden'
                                              )
                        )
            ],
            layout=widgets.Layout(width='400px')
            )
        
        self.output_panel = widgets.VBox([
            self.output_image,
            widgets.Box([self.output_hist],
                        layout=widgets.Layout(overflow_x='hidden',
                                              overflow_y='hidden'
                                              )
                        )
            ],
            layout=widgets.Layout(width='400px'))
        
        self.UI = widgets.HBox([
            self.controls,
            self.input_panel,
            self.output_panel
            ],
            layout=widgets.Layout(height='520px')
            )

    # Funciones que definen el comportamiento de los widgets
    def initLoadButton(self):
        
        self.load_button = widgets.FileUpload(
            accept='',
            multiple=False,
            description="Abrir jpg"
            )
        
        self.load_button.observe(self.updateInputImage, "value")
        self.load_button.observe(self.updateInputHist, "value")

    def initSaveButton(self):

        self.save_button = widgets.Button(description="Guardar Imagen")

        self.save_button.on_click(self.saveImage)

    def initRunButton(self):

        self.run_button = widgets.Button(description="Aplicar")

        self.run_button.on_click(self.updateOutputImage)
        self.run_button.on_click(self.updateOutputHist)

    def initHistDropdown(self):

        self.hist_dropdown = widgets.Dropdown(
            options=[('5', 5), ('10', 10), ('15', 15), ('20', 20), ('25', 25)],
            value=10,
            continuous_update=True
            )

        self.hist_dropdown.observe(self.updateHistDropdown, "value")
        self.hist_dropdown.observe(self.updateInputHist, "value")
        self.hist_dropdown.observe(self.updateOutputHist, "value")

        self.updateHistDropdown(None)
    
    def updateHistDropdown(self, change):

        self.par['hist_bins'] = self.hist_dropdown.value
    
    def initFuncDropdown(self):

        self.func_dropdown = widgets.Dropdown(
            options=["Potencia",
                     "Lineal a trozos",
                     "Multiplicación por escalar"],
                     continuous_update=True
                     )
        
        self.func_dropdown.observe(self.updateFunc, "value")
        self.func_dropdown.observe(self.plotFunction, "value")

        self.updateFunc(None)

    def updateFunc(self, change):

        if self.func_dropdown.value == "Lineal a trozos":
            self.slider.children = [self.y_range_slider]
            self.par['func_dropdown'] = "Lineal a trozos"
        elif self.func_dropdown.value == "Potencia":
            self.slider.children = [self.exp_slider]
            self.par['func_dropdown'] = "Potencia"
        elif self.func_dropdown.value == "Multiplicación por escalar":
            self.slider.children = [self.scalar_slider]
            self.par['func_dropdown'] = "Multiplicación por escalar"
        else:
            pass

    def initLinear(self):

        self.y_range_slider = widgets.FloatRangeSlider(
            description="Rango:", 
            min=0, max=1, step=0.1, value=[0.2, 0.8],
            continuous_update=True
            )

        self.y_range_slider.observe(self.updateLinear, "value")
        self.y_range_slider.observe(self.plotFunction, "value")

        self.updateLinear(None)

    def updateLinear(self, change):

        self.par['y_range'] = self.y_range_slider
        
    def initPower(self):

        self.exp_slider = widgets.FloatSlider(
            description="Exponente:", 
            min=0, max=3, step=0.25, value=2,
            continuous_update=True
            )

        self.exp_slider.observe(self.updatePower, "value")
        self.exp_slider.observe(self.plotFunction, "value")

        self.updatePower(None)

    def updatePower(self, change):

        self.par['exp'] = self.exp_slider.value

    def initScalarMult(self):

        self.scalar_slider = widgets.FloatSlider(
            description="Factor escalar:", 
            min=0.25, max=4, step=0.25, value=1.5,
            continuous_update=True
            )

        self.scalar_slider.observe(self.updateScalar, "value")
        self.scalar_slider.observe(self.plotFunction, "value")

        self.updateScalar(None)

    def updateScalar(self, change):

        self.par['scalar'] = self.scalar_slider.value

    def initFuncOutput(self):

        self.func_output = widgets.Output(layout=widgets.Layout(
            width='280px',
            height='280px'
            )
        )
        
        self.plotFunction(None)    

    def plotFunction(self, change):

        self.function = {
            "Lineal a trozos": lambda x: piecewise_linear(x,
                                                  self.par['y_range'].value[0],
                                                  self.par['y_range'].value[1]
                                                  ),
            "Potencia": lambda x: x**self.par['exp'],
            "Multiplicación por escalar": lambda x: self.par['scalar']*x
            }

        with self.func_output:
            ipd.clear_output() 

            xvals = np.linspace(0, 1, 100)
            yvals = list(map(self.function[self.par['func_dropdown']], xvals))

            fig, ax = plt.subplots()
            plt.plot(xvals, yvals, linestyle='solid', color="black", linewidth=3)
            plt.grid()

            figFormat (fig, ax, 360, 360, 2, 'black')
            
            ax.set_xlabel('Luminancia de entrada', fontsize=14, fontweight='bold')
            ax.set_ylabel('Luminancia de salida', fontsize=14, fontweight='bold')

            plt.xlim([0, 1]), plt.ylim([0, 1]), ax.set_aspect('equal', 'box')
            plt.xticks(weight='bold')
            plt.yticks(weight='bold')
            plt.show()

    def initInputImage(self):
        
        self.input_image = widgets.Box(layout = widgets.Layout(
            overflow_x='hidden',
            overflow_y='hidden',
            width='396px',
            height='320px',
            justify_content='center',
            align_items='center'
            )
        )
    
    def updateInputImage (self, change):

        self.input_image.children = [widgets.Image(
            value=self.load_button.data[0],
            format='jpg',
            layout = widgets.Layout(
                object_fit = 'contain',
                overflow_x='hidden',
                overflow_y='hidden'
                )
            )
        ]

    def initOutputImage(self):

        self.output_image = widgets.Box(layout = widgets.Layout(
            overflow_x='hidden',
            overflow_y='hidden',
            width='396px',
            height='320px',
            justify_content='center',
            align_items='center'
            )
        )
    
    def updateOutputImage(self, change):

        try:
            img = np.array(Image.open(BytesIO(self.load_button.data[0])))
            img_yiq = rgb_to_yiq(img)

            y_in = img_yiq[:,:,0]
            y_out=self.function[self.par['func_dropdown']](y_in)

            img_yiq_out = np.dstack((y_out,
                                     img_yiq[:,:,1],
                                     img_yiq[:,:,2]
                                     ))
            
            img_out = yiq_to_rgb(img_yiq_out)

            # paso la imagen a bytes para widgets.Image
            img_out = Image.fromarray(np.uint8(img_out))
            buf = BytesIO()
            img_out.save(buf, format='JPEG')
            self.bytes_img = buf.getvalue()

            self.output_image.children = [widgets.Image(
                value=self.bytes_img,
                format='jpg',
                layout = widgets.Layout(object_fit = 'contain',
                                        overflow_x='hidden',
                                        overflow_y='hidden'
                                        )
                )]

            self.par["output_image"] = self.bytes_img
        except:
            pass
    
    def initInputHist(self):

        self.input_hist = widgets.Output(layout=widgets.Layout(height='180px'))

    def updateInputHist (self, change):

        try:
            self.input_hist_fig,_ = histogram(self.load_button.data[0],
                                              self.par['hist_bins']
                                              )
            with self.input_hist:
                ipd.clear_output()
                display(self.input_hist_fig)
        except:
            pass

    def initOutputHist(self):

        self.output_hist = widgets.Output(layout=widgets.Layout(height='180px'))

    def updateOutputHist (self, change):

        try:
            self.output_hist_fig,_ = histogram(self.par["output_image"],
                                               self.par['hist_bins']
                                               )
            with self.output_hist:
                ipd.clear_output()
                display(self.output_hist_fig)
        except:
            pass

    def saveImage(self,change):

        try:
            # Transformo el array de numpy en una imagen de PIL y la guardo
            img_to_save = Image.open(BytesIO(self.par["output_image"]))
            output_filename = 'new_image.jpg'
            img_to_save.save(output_filename)

            print('La imagen ha sido guardada como ' + '\'' + output_filename + '\'.')
        except:
            pass

    def _ipython_display_(self):
        display(self.UI)

ModelUI()

#Observaciones:


* La imagen `imagen_oscura.jpg` tiene el problema de que no se aprecian correctamente los detalles en los pinos de la parte inferior.
 Viendo el histograma de luminancia se ve que hay una acumulación en los extremos del mismo, lo que implica una imagen con muchos píxeles muy oscuros, muchos píxeles muy luminosos y pocos con luminancia intermedia.

 Al seleccionar la operación `Multiplicación por escalar` con un escalar mayor que 1, la luminancia de todos los píxeles aumenta de la misma proporción, lo cual produce una imagen de salida más luminosa, pero las regiones más claras terminan saturando al blanco, lo cual destruye parte de la información de la imagen.

 Una mejor solución es usar una función potencia, elevando el valor de luminancia de cada pixel a un exponente menor que 1 (por ejemplo 0.5 o 0.75). Al hacer esto (seleccionando la función `Potencia`) la imagen gana luminancia, pero el cambio en este caso es mayor en los píxeles más oscuros, con lo que se pueden apreciar mejor todos los detalles. Se ve que en el histograma de la imagen resultante la primer barra se dispersa, o sea que los píxeles mas oscuros se distribuyen entre valores con mayor luminancia.


* La imagen `imagen_clara.jpg` es muy luminosa en general, y sobre todo alrededor del Sol. Se puede ver que los píxeles se acumulan en el extremo derecho del histograma de luminancia.

 Al multiplicar la luminancia por un escalar menor que 1 la imagen se oscurece de forma homogénea, lo cual no mejora su apariencia.

 La solución es aplicar una función potencia con exponente mayor que 1 (con 3 queda bastante bien), con lo cual la imagen adquiere un aspecto más natural.

 Viendo el histograma de la imagen resultante se ve que los píxeles se acumulan donde la función tiene pendiente menor que 1 (entre Y=0 e Y=0.6 aproximadamente), y la región con pendiente mayor que 1 dispersa los píxeles.
 

* La imagen `poco_rango_dinamico.jpg` tiene poco contraste. No está muy iluminada ni muy oscura, sino que todos sus píxeles tienen valores similares de luminancia. Esto se comprueba viendo el histograma, donde se aprecia que su rango dinámico está acotado al intervalo `[0.4, 0.8]`, con lo cual se puede mejorar aplicando la función `Lineal a trozos` y seleccionando el rango `[0.4, 0.8]`.

 La imagen resultante es mucho más nítida y el rango dinámico abarca más valores de luminancia.